# Assignment

In this assignment we will train a contracting-expanding convolutional neural network (CNN) to perform semantic segmentation of the prostate gland on MRI. Specifically, this algorithm will be implemented as a three-class classifier: background; transitional zone of the prostate; peripheral zone of the prostate. The ability to properly capture this anatomic context is a critical first step in characterizing a potential prostate lesion.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/cnn/assignment.ipynb)
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [1]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [4]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 276kB 7.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=ddb17bdbb47dfb2df93abb3f032b8ef26c2e4153795a6a6cb8a29b477b966cde
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets, custom

# Data

As in the tutorial, data for this assignment will consist of prostate MRI exams. In prior work, an algorithm was created to separate out different MRI sequences. In this current assignment, only T2-weighted images (isolated using the prior algorithm) will be used for segmentation. In prostate imaging, the T2-weighted sequence captures the greatest amount of anatomic detail and is thus ideal for delineation of prostate gland structures.

The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [6]:
# --- Download dataset
datasets.download(name='mr/prostatex-seg')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-256')
inputs = client.get_inputs(Input)

[ 2020-06-03 23:50:45 ] [====================] 100.000% : Extracting archive (0001381 / 0001381) 

# Training

In this assignment we will train a standard 2D U-Net for prostate segmentation. In addition to the baseline U-Net architecture, at minimum the following modifications must be implemented:

* same padding (vs. valid padding)
* strided convolutions (vs. max-pooling)

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [0]:
# --- Define model
xs, ys = next(gen_train)
inputs = client.get_inputs(Input)
conv = lambda x, filters, strides : layers.Conv3D(filters = filters, strides = strides, kernel_size = (1,3,3), padding = 'same')(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x : relu(norm(conv(x,filters, strides = (1,2,2))))
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters = filters, strides = strides, padding = 'same', kernel_size= (1,3,3))(x)
tran2 = lambda filters, x: relu(norm(tran(x, filters, strides = (1,2,2))))
concat = lambda a,b: layers.Concatenate()([a,b])
l6 = tran2(48, l5)
l7 = tran2(32, conv1(48, concat(l4, l6)))
l8 = tran2(16, conv1(32, concat(l3, l7)))
l9 = tran2(8, conv1(16, concat(l2, l8)))
l10 = conv1(8, l9)

# --- Create logits
logits = {}
logits['zones'] = layers.Conv3D(filters=3, name='zones', padding = 'same', kernel_size=(1,3,3))(l10)

# # --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile the model

In [0]:
# --- Compile model
model.compile(
    optimizer = optimizers.Adam(learning_rate=2e-4),
    loss = {'zones': losses.SparseCategoricalCrossentropy(from_logits = True)},
    metrics = {'zones': custom.dsc(cls=2)},
    experimental_run_tf_function=False                                                    
)

### Train the model

In [25]:
model.fit(
    x = gen_train,
    steps_per_epoch = 500,
    epochs = 4,
    validation_data = gen_valid,
    validation_steps = 500,
    validation_freq = 4,
    use_multiprocessing = True
)

Epoch 1/4
500/500 [==============================] - 60s 119ms/step - loss: 0.5463 - dsc_1: 0.2816 - dsc_2: 0.0022
Epoch 2/4
500/500 [==============================] - 61s 122ms/step - loss: 0.0812 - dsc_1: 0.7380 - dsc_2: 0.1185
Epoch 3/4
500/500 [==============================] - 61s 121ms/step - loss: 0.0396 - dsc_1: 0.8294 - dsc_2: 0.5944
Epoch 4/4
500/500 [==============================] - 114s 228ms/step - loss: 0.0279 - dsc_1: 0.8596 - dsc_2: 0.6516 - val_loss: 0.0273 - val_dsc_1: 0.8640 - val_dsc_2: 0.6560


# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

The Dice score values should be calculated both for peripheral and transitional zone (class 1 and 2); the Dice score for background does not need to be evaluated. As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

* peripheral zone: mean Dice score > 0.75
* transitional zone: mean Dice score > 0.55

In [0]:
def dice(y_true, y_pred, c=1, epsilon=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      (np.ndarray) y_true : ground-truth label
      (np.ndarray) y_pred : predicted logits scores
      (int)             c : class to calculate DSC on
    
    """
    assert y_true.ndim == y_pred.ndim
    
    true = y_true[..., 0] == c
    pred = np.argmax(y_pred, axis=-1) == c 

    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred) + epsilon
    
    return A / B

In [27]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_pz = []
dsc_tz = []

for x, y in test_valid:
    logits = model.predict(x['dat'])
    if type(logits) is dict:
      logits = logits['zones']
      dsc_pz.append(dice(y['zones'][0], logits[0], c=1))
      dsc_tz.append(dice(y['zones'][0], logits[0], c=2))
    pass
dsc_pz = np.array(dsc_pz)
dsc_tz = np.array(dsc_tz)

[ 2020-06-04 00:20:43 ] [====================] 100.000% : Iterating | 000069    

**Note**: this cell is used only to check for model performance prior to submission. It will not be graded. Once submitted, your model will be benchmarked against the (same) validation cohort to determine final algorithm performance and grade. If your evaluation code above is correct the algorithm accuracy should match and you can confident that you will recieve full credit for the assignment. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort Dice score statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least two columns in the `*.csv` file:

* Dice score (transitional zone)
* Dice score (peripheral zone)

In [29]:
# --- Create *.csv
df = pd.DataFrame(index = np.arange(dsc_tz.size))
df['dsc_tz'] = dsc_tz
df['dsc_pz'] = dsc_pz
print(df['dsc_pz'].mean())
print(df['dsc_tz'].mean())

# --- Serialize *.csv
fname = '{}/models/organ_segmentation/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
# f.to_csv(path)

0.8597681439135537
0.6575380500585714


# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [0]:
# --- Serialize a model
fname = '{}/models/organ_segmentation/final.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.